In [25]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import csv

class ArtikelApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Biker Applicatie")
        self.root.geometry("850x400")
        self.root.resizable(False, False)

        # Frame voor mijn lijst
        self.artikelen_frame = ttk.Frame(self.root)
        self.artikelen_frame.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")

        # Treeview voor het tonen van artikelen
        self.artikelen_tree = ttk.Treeview(self.artikelen_frame, columns=("ID", "Nummer", "Naam", "Type", "Prijs", "Voorraad", "Aanwezig", "Aanschafdatum"))
        self.artikelen_tree.heading("#0", text="", anchor=tk.W)  # Verwijder de lege kop aan de rechterkant
        self.artikelen_tree.heading("ID", text="ID", anchor=tk.W)
        self.artikelen_tree.heading("Nummer", text="Nummer", anchor=tk.W)
        self.artikelen_tree.heading("Naam", text="Naam", anchor=tk.W)
        self.artikelen_tree.heading("Type", text="Type", anchor=tk.W)
        self.artikelen_tree.heading("Prijs", text="Prijs", anchor=tk.W)
        self.artikelen_tree.heading("Voorraad", text="Voorraad", anchor=tk.W)
        self.artikelen_tree.heading("Aanwezig", text="Aanwezig", anchor=tk.W)
        self.artikelen_tree.heading("Aanschafdatum", text="Aanschafdatum", anchor=tk.W)

        # De kolombreedtes
        self.artikelen_tree.column("#0", width=0, stretch=tk.NO)  # Verwijder de lege kolom
        self.artikelen_tree.column("ID", width=80)
        self.artikelen_tree.column("Nummer", width=80)
        self.artikelen_tree.column("Naam", width=200)
        self.artikelen_tree.column("Type", width=100)
        self.artikelen_tree.column("Prijs", width=80)
        self.artikelen_tree.column("Voorraad", width=80)
        self.artikelen_tree.column("Aanwezig", width=80)
        self.artikelen_tree.column("Aanschafdatum", width=120)

        self.artikelen_tree.pack(side="left", fill="both")

        # Scrollbar
        self.artikelen_scrollbar = ttk.Scrollbar(self.artikelen_frame, orient="vertical", command=self.artikelen_tree.yview)
        self.artikelen_scrollbar.pack(side="right", fill="y")
        self.artikelen_tree.configure(yscrollcommand=self.artikelen_scrollbar.set)

        # Knoppen
        self.importeer_knop = ttk.Button(self.root, text="Importeer CSV", command=self.importeer_csv)
        self.importeer_knop.grid(row=1, column=0, padx=10, pady=10)

        self.verwijder_knop = ttk.Button(self.root, text="Verwijder Geselecteerde", command=self.verwijder_geselecteerde)
        self.verwijder_knop.grid(row=1, column=0, padx=(10,0), pady=10, sticky='w')

        self.bewerk_knop = ttk.Button(self.root, text="Bewerk Geselecteerde", command=self.bewerk_geselecteerde)
        self.bewerk_knop.grid(row=2, column=0, padx=(10,0), pady=10, sticky='w')

        self.toevoegen_knop = ttk.Button(self.root, text="Voeg Nieuw Artikel Toe", command=self.toevoeg_nieuw_artikel)
        self.toevoegen_knop.grid(row=3, column=0, padx=(10,0), pady=10, sticky='w')

        self.artikel_id_counter = 1  # Houdt de laatste toegewezen ID bij

        self.afsluiten_knop = ttk.Button(self.root, text="Afsluiten", command=self.afsluiten_programma)
        self.afsluiten_knop.grid(row=3, column=0, padx=(0,20), pady=10, sticky='e')
        

    def afsluiten_programma(self):
        self.root.destroy()

    def importeer_csv(self):
        bestandsnaam = filedialog.askopenfilename(filetypes=[("CSV-bestanden", "*.csv")])
        if bestandsnaam:
            # Lees de CSV en voeg de gegevens toe aan de Treeview
            with open(bestandsnaam, 'r') as csv_file:
                csv_reader = csv.DictReader(csv_file, delimiter=';')
                for row in csv_reader:
                    self.artikelen_tree.insert("", "end", values=(
                        self.artikel_id_counter,
                        row["artikel nummer"],
                        row["artikel naam"],
                        row["artikel type"],
                        row["artikel prijs"],
                        row["artikel voorraad"],
                        row["artikel aanwezigheid"],
                        row["aanschaf datum"]
                    ))
                    self.artikel_id_counter += 1

    def verwijder_geselecteerde(self):
        geselecteerde_items = self.artikelen_tree.selection()
        for item in geselecteerde_items:
            self.artikelen_tree.delete(item)

    def bewerk_geselecteerde(self):
        geselecteerde_items = self.artikelen_tree.selection()
        if geselecteerde_items:
            # Haal de waarden op van het eerste geselecteerde item 
            geselecteerd_item = geselecteerde_items[0]
            waarden = self.artikelen_tree.item(geselecteerd_item, 'values')

            # Open een nieuw venster om bewerkingen uit te voeren
            bewerk_venster = tk.Toplevel(self.root)
            bewerk_venster.title("Bewerk Artikel")

            # Voeg invoervakken toe voor het bewerken van de waarden
            labels = ["Nummer", "Naam", "Type", "Prijs", "Voorraad", "Aanwezig", "Aanschafdatum"]
            entry_widgets = []

            for label, waarde in zip(labels, waarden[1:]):  # Start vanaf index 1 om de echte ID over te slaan
                label_widget = ttk.Label(bewerk_venster, text=label)
                entry_widget = ttk.Entry(bewerk_venster)

                entry_widget.insert(0, waarde)

                label_widget.grid(row=len(entry_widgets), column=0, padx=5, pady=5)
                entry_widget.grid(row=len(entry_widgets), column=1, padx=5, pady=5)

                entry_widgets.append(entry_widget)

            # knoppen voor opslaan en annuleren
            opslaan_knop = ttk.Button(bewerk_venster, text="Opslaan", command=lambda: self.opslaan_bewerking(geselecteerd_item, entry_widgets, bewerk_venster))
            annuleren_knop = ttk.Button(bewerk_venster, text="Annuleren", command=bewerk_venster.destroy)

            opslaan_knop.grid(row=len(entry_widgets), column=0, padx=5, pady=5)
            annuleren_knop.grid(row=len(entry_widgets), column=1, padx=5, pady=5)

    def toevoeg_nieuw_artikel(self):
        # nieuw venster om een nieuw artikel toe te voegen
        toevoeg_venster = tk.Toplevel(self.root)
        toevoeg_venster.title("Voeg Nieuw Artikel Toe")

        # Voeg invoervakken toe voor het toevoegen van het artikel
        labels = ["Nummer", "Naam", "Type", "Prijs", "Voorraad", "Aanwezig", "Aanschafdatum"]
        entry_widgets = []

        for label in labels:
            label_widget = ttk.Label(toevoeg_venster, text=label)
            entry_widget = ttk.Entry(toevoeg_venster)

            label_widget.grid(row=len(entry_widgets), column=0, padx=5, pady=5)
            entry_widget.grid(row=len(entry_widgets), column=1, padx=5, pady=5)

            entry_widgets.append(entry_widget)

        # Knoppen  voor toevoegen en annuleren
        toevoegen_knop = ttk.Button(toevoeg_venster, text="Voeg Toe", command=lambda: self.voeg_nieuw_toe(entry_widgets, toevoeg_venster))
        annuleren_knop = ttk.Button(toevoeg_venster, text="Annuleren", command=toevoeg_venster.destroy)

        toevoegen_knop.grid(row=len(entry_widgets), column=0, padx=5, pady=5)
        annuleren_knop.grid(row=len(entry_widgets), column=1, padx=5, pady=5)

    def voeg_nieuw_toe(self, entry_widgets, toevoeg_venster):
        nieuwe_waarden = [entry.get() for entry in entry_widgets]
        self.artikelen_tree.insert("", "end", values=nieuwe_waarden)
        self.artikel_id_counter += 1
        toevoeg_venster.destroy()

    def opslaan_bewerking(self, geselecteerd_item, entry_widgets, bewerk_venster):
        oude_waarden = list(self.artikelen_tree.item(geselecteerd_item, 'values'))
        nieuwe_waarden = [entry.get() for entry in entry_widgets]

        for index, nieuwe_waarde in enumerate(nieuwe_waarden):
            oude_waarden[index + 1] = nieuwe_waarde

        self.artikelen_tree.item(geselecteerd_item, values=tuple(oude_waarden))
        bewerk_venster.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = ArtikelApp(root)
    root.mainloop()









